# Import necessary libraries

In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os, sys
from functools import reduce

from sklearn.preprocessing import *
from multiprocessing import Pool
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import *

from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.naive_bayes import *

os.chdir("/home/sieu/PycharmProjects/ML-Vincent-Ng/prj_new")

# Training import

In [2]:
column_name = pd.read_table('attr.txt', sep=":", usecols=all, names = ['attr', 'range'])
df = pd.read_table('train.txt', sep="\s+", usecols=all, names = list(column_name['attr']))

In [3]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [ ]:
df.hist(figsize=(90, 90))
plt.show()

# Preproccessing

In [15]:
# Seperate class from attributes
y = df["Class"].to_numpy()
X = df[df.columns[:-1]]
attr_lst = X.columns

In [16]:
# Remove all constant-valued features
sel = VarianceThreshold()
sel.feature_names_in_= attr_lst
X=sel.fit_transform(X)
attr_lst=sel.get_feature_names_out(attr_lst)
print(attr_lst)
print("Number of features: ", len(attr_lst))
print(X.shape)

['B1' 'B2' 'B3' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'C7' 'C8' 'C9' 'C10' 'C11'
 'C12' 'C13' 'C14' 'C15' 'C16' 'C17' 'C18' 'C19' 'C20' 'C21' 'C22' 'C23'
 'C24' 'C25' 'C26' 'C27' 'C28' 'C29' 'C30' 'C31' 'C32' 'C33' 'C34' 'C35'
 'C36' 'C37' 'C38' 'C39' 'C40' 'C41' 'C42' 'C43' 'C44' 'C45' 'C46' 'C47'
 'C48' 'C49' 'C50' 'C51' 'C52' 'C53' 'C54' 'C55' 'C56' 'C57' 'C58' 'C59'
 'C60' 'C61' 'C62' 'C63' 'C64' 'C65' 'C66' 'C67' 'C68' 'C69' 'C70' 'C71'
 'C72' 'C73' 'C74' 'C75' 'C76' 'C77' 'C78' 'C79' 'C80' 'C81' 'C82' 'C83'
 'C84' 'C85' 'C86' 'C87' 'C88' 'C89' 'C90' 'C91' 'C92' 'C93' 'C94' 'C95'
 'C96' 'C97' 'C98' 'C99' 'C100' 'C101' 'C102' 'C103' 'C104' 'C105' 'C106'
 'C107' 'C108' 'C109' 'C110' 'C111' 'C112' 'C113' 'C114' 'C115' 'C116'
 'C117' 'C118' 'C119' 'C120' 'C121' 'C122' 'C123' 'C124' 'C125' 'C126'
 'C127' 'C128' 'C129' 'C130' 'C131' 'C132' 'C133' 'C134' 'C135' 'C136'
 'C137' 'C138' 'C139' 'YEAR' 'C140' 'C141' 'C142' 'CT1' 'CT2' 'CT3' 'CT4'
 'CT5' 'CT6' 'CT9' 'CT10' 'CT11' 'CT12' 'CT13' 'CT14' '

In [ ]:
X_bin

# Select good features

In [17]:
n_fet_to_sel = len(attr_lst)//3

# print("BEGIN: BernoulliNB classifier feature selection")
# berNB = SequentialFeatureSelector(estimator=BernoulliNB(),n_features_to_select=n_fet_to_sel, direction="forward").fit(X, y)
# berNB.feature_names_in_= attr_lst
# attr_lst = berNB.get_feature_names_out(attr_lst)
# print(attr_lst)
# print("Done BernoulliNB classifier feature selection")

print("BEGIN: HistGrad classifier feature selection")
berNB = SequentialFeatureSelector(estimator=HistGradientBoostingClassifier(max_leaf_nodes=60,max_iter=3000,learning_rate=0.06,l2_regularization=0.15, max_depth=8, max_bins=24, early_stopping=True, random_state=0),n_features_to_select=n_fet_to_sel, direction="forward").fit(X, y)
berNB.feature_names_in_= attr_lst
attr_lst = berNB.get_feature_names_out(attr_lst)
print(attr_lst)
print("Done HistGrad classifier feature selection")

BEGIN: BernoulliNB classifier feature selection
['B1' 'B2' 'C1' 'C2' 'C3' 'C4' 'C6' 'C7' 'C8' 'C10' 'C11' 'C15' 'C23'
 'C26' 'C28' 'C31' 'C41' 'C46' 'C56' 'C71' 'C76' 'C86' 'C101' 'C116'
 'C131' 'C136' 'C137' 'C138' 'C139' 'YEAR' 'C140' 'C141' 'C142' 'CT1'
 'CT2' 'CT3' 'CT4' 'CT5' 'CT6' 'CT9' 'CT10' 'CT11' 'CT12' 'CT13' 'CT14'
 'CT15' 'CT16' 'CT17' 'CT18' 'CT19' 'CT20' 'CT21' 'CT22' 'CT23' 'CT24'
 'CT25' 'CT26']
Done BernoulliNB classifier feature selection


In [21]:
# Save feature selection results
with open("bernolli_fet_1,3.pkl", 'wb') as file:
    pickle.dump(attr_lst, file)

# Load good feature

In [61]:
# Load good feature list
with open("bernolli_fet_1,3.pkl", 'rb') as file:
    attr_lst = pickle.load(file)
print(attr_lst)
print(len(attr_lst))

['B1' 'B2' 'C1' 'C2' 'C3' 'C4' 'C6' 'C7' 'C8' 'C10' 'C11' 'C15' 'C23'
 'C26' 'C28' 'C31' 'C41' 'C46' 'C56' 'C71' 'C76' 'C86' 'C101' 'C116'
 'C131' 'C136' 'C137' 'C138' 'C139' 'YEAR' 'C140' 'C141' 'C142' 'CT1'
 'CT2' 'CT3' 'CT4' 'CT5' 'CT6' 'CT9' 'CT10' 'CT11' 'CT12' 'CT13' 'CT14'
 'CT15' 'CT16' 'CT17' 'CT18' 'CT19' 'CT20' 'CT21' 'CT22' 'CT23' 'CT24'
 'CT25' 'CT26']
57


In [ ]:
# Load from clipboard
attr_lst = ['B1', 'B3', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8', 'C12', 'C14',
       'C23', 'C32', 'C39', 'C40', 'C41', 'C45', 'C46', 'C51', 'C54',
       'C77', 'C91', 'C92', 'C99', 'C101', 'C104', 'C105', 'C111', 'C116',
       'C137', 'C141', 'CT18', 'CT19', 'CT22']
attr_lst=np.array(attr_lst)

In [31]:
# Best model validation
X_new=df[attr_lst].to_numpy()
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle=True)
arr = []
#clf = BernoulliNB()
clf = HistGradientBoostingClassifier(max_leaf_nodes=60,max_iter=3000,learning_rate=0.06,l2_regularization=0.15, max_depth=8, max_bins=24, early_stopping=True)
for train_index , test_index in kfold.split(X_new):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    arr.append(clf.score(X_test, y_test))
print("Done")
print("mean acc: ", np.mean(arr))

0.7597998331943286
0.7297748123436196
0.7562604340567612
0.7278797996661102
0.7454090150250418
0.7353923205342237
0.7487479131886478
0.7454090150250418
0.7545909849749582
0.7504173622704507
Done
mean acc:  0.7453681490279183


In [25]:
attr_lst

array(['B1', 'B2', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8', 'C10', 'C11',
       'C15', 'C23', 'C26', 'C28', 'C31', 'C41', 'C46', 'C56', 'C71',
       'C76', 'C86', 'C101', 'C116', 'C131', 'C136', 'C137', 'C138',
       'C139', 'YEAR', 'C140', 'C141', 'C142', 'CT1', 'CT2', 'CT3', 'CT4',
       'CT5', 'CT6', 'CT9', 'CT10', 'CT11', 'CT12', 'CT13', 'CT14',
       'CT15', 'CT16', 'CT17', 'CT18', 'CT19', 'CT20', 'CT21', 'CT22',
       'CT23', 'CT24', 'CT25', 'CT26'], dtype=object)

# GridSearch for best model

In [ ]:
# Naive PCA
params = {'max_leaf_nodes':np.arange(200,500, 20), 'l2_regularization':np.arange(0.5,1,0.05), 'learning_rate':np.arange(0.3,1,0.05), 'max_depth':np.arange(8, 40, 2), 'max_bins':np.arange(20, 50, 2)}
#params = {'max_iter':np.arange(50,200,100),'max_leaf_nodes':np.arange(100,500, 10), 'l2_regularization':np.arange(0,1,0.05)}
search = GridSearchCV(estimator=HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), 
                                param_grid=params).fit(X_new,y)

In [26]:
# Normal Grid
params = {'max_leaf_nodes':np.arange(20,90, 10), 'l2_regularization':np.arange(0.05,0.4,0.05), 'learning_rate':np.arange(0.01,0.1,0.01), 'max_depth':np.arange(6, 12, 1), 'max_bins':np.arange(16, 34, 2)}
#params = {'max_iter':np.arange(50,200,100),'max_leaf_nodes':np.arange(100,500, 10), 'l2_regularization':np.arange(0,1,0.05)}
search = GridSearchCV(estimator=HistGradientBoostingClassifier(max_iter=3000, early_stopping=True), 
                                param_grid=params, n_jobs=-1).fit(X_new,y)

In [ ]:
attr_lst

In [27]:
search.score(X_new, y)

0.8158905024202971

In [28]:
# Save models
with open("815_1,3_best.pkl", 'wb') as file:
    pickle.dump(search, file)

In [ ]:
# Load model
with open("80_best.pkl", 'rb') as file:
    search = pickle.load(file)

In [ ]:
search.score(X_new, y)

In [29]:
search.best_params_

{'l2_regularization': 0.15000000000000002,
 'learning_rate': 0.060000000000000005,
 'max_bins': 24,
 'max_depth': 8,
 'max_leaf_nodes': 60}

In [ ]:
# Best model validation
kfold = sklearn.model_selection.KFold(n_splits=3, shuffle=True)
arr = []
#clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2, 4, 5, 6], max_leaf_nodes=50,max_iter=3000,learning_rate=0.06,l2_regularization=0.25, max_depth=11, max_bins=42, early_stopp)
for train_index , test_index in kfold.split(X_new):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #clf.fit(X_train, y_train)
    #print(clf.score(X_test, y_test))
    #arr.append(clf.score(X_test, y_test))
    arr.append(search.score(X_test, y_test))
    print(search.score(X_test, y_test))
print("Done")
print("mean acc: ", np.mean(arr))

# Essemble

In [ ]:
def Ridge_classifiers(data):
    clf = RidgeClassifier(solver="svd")
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def knn_classifiers(data):
    clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def svc_classifiers(data):
    clf = SVC(kernel='rbf', degree=3)
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def BernoulliNB_classifiers(data):
    clf = BernoulliNB()
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def HGB_classifiers(data):
    clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2])
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)

In [ ]:
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle=True)
dataset = []
for train_index , test_index in kfold.split(X):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    dataset.append((X_train, y_train, X_test, y_test))
    #Testing individual clf
    clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2], max_leaf_nodes=800,max_iter=300,learning_rate=0.8500000000000002,l2_regularization=0.8500000000000001).fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
    #clf = RidgeClassifier(solver="svd").fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
    #clf = BernoulliNB().fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
print("Done")

In [ ]:
assembly = []
vote_weights = []
def append_results(result):
    for clf, score in result:
        assembly.append(clf)
        vote_weights.append(score)
if __name__ == '__main__':
    p = Pool()
    p.map_async(Ridge_classifiers, dataset, callback=append_results)
    #p.map_async(nn_classifiers, dataset, callback=append_results)
    #p.map_async(knn_classifiers, dataset, callback=append_results)
    #p.map_async(svc_classifiers, dataset, callback=append_results)
    p.map_async(BernoulliNB_classifiers, dataset, callback=append_results)
    p.map_async(HGB_classifiers, dataset, callback=append_results)
    p.close()
    p.join()

In [ ]:
print(assembly)
print(vote_weights)

In [ ]:
clfs = []
for i in range(len(assembly)):
    clfs.append(("Classifier " + str(i), assembly[i]))
eclf = VotingClassifier(estimators=clfs,voting='hard', weights=vote_weights, n_jobs=-1)

In [ ]:
cross_val_score(eclf, X_new, y, scoring='accuracy', cv=10, n_jobs=-1)

# Evaluation 

In [81]:
attr = pd.read_table('attr.txt', sep=":", usecols=all, names = ['attr', 'range'])
dfX_pred = pd.read_table('prelim.txt', sep="\s+", usecols=all, names = list(attr['attr']))
dfX_pred = dfX_pred.dropna()

In [82]:
with open("bernolli_fet_1,3.pkl", 'rb') as file:
    attr_lst = pickle.load(file)
list(attr_lst)

['B1',
 'B2',
 'C1',
 'C2',
 'C3',
 'C4',
 'C6',
 'C7',
 'C8',
 'C10',
 'C11',
 'C15',
 'C23',
 'C26',
 'C28',
 'C31',
 'C41',
 'C46',
 'C56',
 'C71',
 'C76',
 'C86',
 'C101',
 'C116',
 'C131',
 'C136',
 'C137',
 'C138',
 'C139',
 'YEAR',
 'C140',
 'C141',
 'C142',
 'CT1',
 'CT2',
 'CT3',
 'CT4',
 'CT5',
 'CT6',
 'CT9',
 'CT10',
 'CT11',
 'CT12',
 'CT13',
 'CT14',
 'CT15',
 'CT16',
 'CT17',
 'CT18',
 'CT19',
 'CT20',
 'CT21',
 'CT22',
 'CT23',
 'CT24',
 'CT25',
 'CT26']

In [90]:
# Load from clipboard
attr_lst = ['B1', 'B2', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8', 'C10', 'C11',
       'C15', 'C23', 'C26', 'C28', 'C31', 'C41', 'C46', 'C56', 'C71',
       'C76', 'C86', 'C101', 'C116', 'C131', 'C136', 'C137', 'C138',
       'C139', 'YEAR', 'C140', 'C141', 'C142', 'CT1', 'CT2', 'CT3', 'CT4',
       'CT5', 'CT6', 'CT9', 'CT10', 'CT11', 'CT12', 'CT13', 'CT14',
       'CT15', 'CT16', 'CT17', 'CT18', 'CT19', 'CT20', 'CT21', 'CT22',
       'CT23', 'CT24', 'CT25', 'CT26']
attr_lst=np.array(attr_lst)

In [91]:
X_pred = dfX_pred[attr_lst].to_numpy()
y_pred = dfX_pred["Class"].to_numpy()

In [92]:
y_pred.shape

(5000,)

In [94]:
# Load model then predict and save prediction
with open("815_1,3_best.pkl", 'rb') as file:
    search = pickle.load(file)
print('The validation score is: ', search.score(X_pred, y_pred))
np.savetxt(fname="prediction.txt", X=search.predict(X_pred), fmt='%d')

The validation score is:  0.7434
